In [1]:
#To supress warnings in the output
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


In [5]:
#dropping stab column as its related with stabf
df=df.drop('stab',axis=1)
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stabf'],
      dtype='object')

In [6]:
#checking for missing values
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

That means no null value

In [7]:
X_data=df.drop('stabf', axis=1)
Y_data=df[['stabf']]

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=1)

In [9]:
#standard scaler

from sklearn.preprocessing import StandardScaler
myscaler=StandardScaler()
Xt_Scaled=pd.DataFrame(myscaler.fit_transform(x_train), columns=x_train.columns)
Xtest_Scaled=pd.DataFrame(myscaler.transform(x_test), columns=x_test.columns)

# Xt_Scaled.columns=x_train.columns
# Xtest_Scaled.columns=x_test.columns

print(Xt_Scaled.head())
print(Xtest_Scaled.head())
# y_train=y_train.reset_index(drop=True)
# y_test=y_test.reset_index(drop=True)
# y_train

       tau1      tau2      tau3  ...        g2        g3        g4
0  0.367327 -0.986042  0.650447  ...  0.339859  0.585568  0.492239
1 -0.064659  0.089437  1.035079  ... -1.558488  1.429649 -1.443521
2 -1.467850  1.298418 -0.502536  ...  1.451534 -1.045743  0.492489
3  0.820081  0.529920  1.299657  ...  1.361958  1.604140  0.275303
4  0.665424 -1.425627  0.312300  ...  0.695660  1.137504 -1.312575

[5 rows x 12 columns]
       tau1      tau2      tau3  ...        g2        g3        g4
0  0.593951 -0.412733  1.503924  ...  1.167203 -1.507330  1.084726
1  0.202190  0.374416 -0.188800  ... -0.395660  1.414651  1.226011
2 -1.079044 -0.313745 -0.884634  ... -1.438495  0.651821 -1.682168
3 -0.083120 -1.107327  0.372805  ... -1.672322 -0.357714  1.055865
4  0.873921  1.438466  0.086662  ... -1.469731  0.956396 -0.819727

[5 rows x 12 columns]


In [10]:
type(x_train)

pandas.core.frame.DataFrame

In [ ]:
#label encoding for the categorical data of column 'stabf'
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train =pd.DataFrame(encoder.fit_transform(y_train.reset_index(drop=True)), columns=y_train.columns)
y_test = pd.DataFrame(encoder.transform(y_test.reset_index(drop=True)),columns=y_test.columns) 

In [12]:
def score_data(y_test,pred_y, clf):
  print('\n\n'+clf)
  print("\n CONFUSION MATRIX \n", confusion_matrix(y_test,pred_y))
  print(classification_report(y_test,pred_y))
  print("Accuracy Score is: ", accuracy_score(y_test, pred_y))

In [13]:
#Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier(n_estimators=100, random_state=1)
Rf.fit(Xt_Scaled,y_train)
pred_y=pd.Series(Rf.predict(Xtest_Scaled))
pred_y.value_counts()

1    1320
0     680
dtype: int64

In [14]:
#Checking the accuracy, confusion matrix etc for Random Forest Classifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
score_data(y_test, pred_y,'RandomForest Classifier')



RandomForest Classifier

 CONFUSION MATRIX 
 [[ 625   87]
 [  55 1233]]
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       712
           1       0.93      0.96      0.95      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Accuracy Score is:  0.929


In [15]:
#XGB Classifier

from xgboost import XGBClassifier
clf=XGBClassifier()
clf.fit(Xt_Scaled,y_train)
pred_y=clf.predict(Xtest_Scaled)
score_data(y_test, pred_y,'XGBClassifier')



XGBClassifier

 CONFUSION MATRIX 
 [[ 603  109]
 [  52 1236]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       712
           1       0.92      0.96      0.94      1288

    accuracy                           0.92      2000
   macro avg       0.92      0.90      0.91      2000
weighted avg       0.92      0.92      0.92      2000

Accuracy Score is:  0.9195


In [16]:
#LGBM Classifier

from lightgbm import LGBMClassifier
clf= LGBMClassifier()
clf.fit(Xt_Scaled,y_train)
pred_y=clf.predict(Xtest_Scaled)
score_data(y_test, pred_y,'LGBM')



LGBM

 CONFUSION MATRIX 
 [[ 635   77]
 [  48 1240]]
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       712
           1       0.94      0.96      0.95      1288

    accuracy                           0.94      2000
   macro avg       0.94      0.93      0.93      2000
weighted avg       0.94      0.94      0.94      2000

Accuracy Score is:  0.9375


In [17]:
#ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier
ext_clf=ExtraTreesClassifier()
ext_clf.fit(Xt_Scaled,y_train)
pred_y=ext_clf.predict(Xtest_Scaled)
score_data(y_test, pred_y, 'ExtraTreesClassifier Initially')



ExtraTreesClassifier Initially

 CONFUSION MATRIX 
 [[ 594  118]
 [  30 1258]]
              precision    recall  f1-score   support

           0       0.95      0.83      0.89       712
           1       0.91      0.98      0.94      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.91      0.92      2000
weighted avg       0.93      0.93      0.92      2000

Accuracy Score is:  0.926


In [18]:
#HyperTuning ExtraTreesClassifier

n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 
hyperparameter_grid = {'n_estimators': n_estimators,'min_samples_leaf': min_samples_leaf,'min_samples_split': min_samples_split,'max_features': max_features}
from sklearn.model_selection import RandomizedSearchCV
etc_clf=ExtraTreesClassifier()
rdm = RandomizedSearchCV(etc_clf, param_distributions = hyperparameter_grid, cv=5, n_iter=10, n_jobs=-1, verbose=1, random_state=1, scoring='accuracy')
rdm.fit(Xt_Scaled, y_train)
rdm.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [19]:
etc_clf_opt=ExtraTreesClassifier(max_features=None,min_samples_leaf=8,min_samples_split=2, n_estimators=1000)
etc_clf_opt.fit(Xt_Scaled, y_train)
pred_y=etc_clf_opt.predict(Xtest_Scaled)
score_data(y_test, pred_y, clf='ExtraTreesClassifier Optimal')



ExtraTreesClassifier Optimal

 CONFUSION MATRIX 
 [[ 618   94]
 [  54 1234]]
              precision    recall  f1-score   support

           0       0.92      0.87      0.89       712
           1       0.93      0.96      0.94      1288

    accuracy                           0.93      2000
   macro avg       0.92      0.91      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Accuracy Score is:  0.926


The Accuracy for Optimal ExtraTrees Classifier is 0.9285, whereas for Untuned ExtratreesClassifier is 0.931
<br>Thus, Optimal Model has lower accuracy than the initial model

---



In [20]:
#Feature Importances for Optimal Model of ExtraTreesClassifier
feature_imp=etc_clf_opt.feature_importances_

xyz=pd.DataFrame(feature_imp, index=Xt_Scaled.columns, columns=['Importance'])
print("Max Importance is of: ", xyz[['Importance']].idxmax())
print("Min Importance is of: ", xyz[['Importance']].idxmin())


Max Importance is of:  Importance    tau2
dtype: object
Min Importance is of:  Importance    p1
dtype: object
